# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time

In [2]:
client_id = "0od3VXrFtzCRgcKL4GtKHA"
api_key = "sVAxWQR2mJyptyIyDRqXhR24NdrRVH3simx7bJSQqc7GXIEl2sj2sB9obPMa5wGPGCJh_jVf-y2dCs9CeVtrooh0GutAGh29c0tsz1mMun0vxLDDw0YQzGPe33KSXHYx"


In [3]:
term = "Cookies"
location = "NYC"
search_limit = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': search_limit
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "H4jJ7XB3CetIr1pg56CczQ", "alias": "levain-bakery-new-york", "name": "Levain Bakery", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/9uUdfbEHUB4D-4s_RVA9zw/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/levain-bakery-new-york?adjust_creative=0od3VXrFtzCRgcKL4GtKHA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=0od3VXrFtzCRgcKL4GtKHA", "review_count": 7388, "categories": [{"alias": "bakeries", "title": "Bakeries"}], "rating": 4.5, "coordinates": {"latitude": 40.7799404643263, "longitude": -73.980282552649}, "transactions": [], "price": "$$", "location": {"address1": "167 W 74th St", "address2": "", "address3": "", "city": "New York", "zip_code": "10023", "country": "US", "state": "NY", "display_address": ["167 W 74th St", "New York, NY 10023"]}, "phone": "+12128746080", "display_phone": "(212) 874-6080", "distance": 8367.197123060287}, {"id": "jz_FnuRpCNzfwwHHQ6Nbeg", "alias": "schmackarys-

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [30]:
def make_api_request(term, location):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key),}
    for i in range(3):
        url_params = {
            'term': term.replace(' ', '+'),
            'location': location.replace(' ', '+'),
            'limit': 50,
            'offset': i * 50
            }
        response = requests.get(url, headers=headers, params=url_params)
    return response

In [31]:
yelp_1 = make_api_request("cookie", "NYC")

In [32]:
yelp_1 = yelp_1.json()

In [33]:
print(yelp_1)

{'businesses': [{'id': 'ae8AqaVJ0sUHT4eJwMq-Pg', 'alias': 'spot-dessert-bar-flushing', 'name': 'Spot Dessert Bar', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/xkqI7T8AnfrEFQ_GyKWOHg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/spot-dessert-bar-flushing?adjust_creative=0od3VXrFtzCRgcKL4GtKHA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=0od3VXrFtzCRgcKL4GtKHA', 'review_count': 498, 'categories': [{'alias': 'desserts', 'title': 'Desserts'}], 'rating': 4.0, 'coordinates': {'latitude': 40.75924, 'longitude': -73.8326}, 'transactions': [], 'price': '$$', 'location': {'address1': '39-16 Prince St', 'address2': '', 'address3': None, 'city': 'Flushing', 'zip_code': '11354', 'country': 'US', 'state': 'NY', 'display_address': ['39-16 Prince St', 'Flushing, NY 11354']}, 'phone': '+19172852278', 'display_phone': '(917) 285-2278', 'distance': 14900.456727810375}, {'id': 'Hs1TH_4FpyzbX6_J5r1XXg', 'alias': 'mollys-cupcakes-new-york', 'name': "Molly's 

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [16]:
df1 = pd.DataFrame.from_dict(yelp_1['businesses'])
# df2 = pd.DataFrame.from_dict(yelp_2['businesses'])
# df3 = pd.DataFrame.from_dict(yelp_3['businesses'])
# df4 = pd.DataFrame.from_dict(yelp_4['businesses'])
# df5 = pd.DataFrame.from_dict(yelp_5['businesses'])
# df6 = pd.DataFrame.from_dict(yelp_6['businesses'])
# df = pd.concat([df1, df2, df3, df4, df5, df6])
df1

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,levain-bakery-new-york,"[{'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 40.7799404643263, 'longitude': -7...",(212) 874-6080,8367.197123,H4jJ7XB3CetIr1pg56CczQ,https://s3-media2.fl.yelpcdn.com/bphoto/9uUdfb...,False,"{'address1': '167 W 74th St', 'address2': '', ...",Levain Bakery,+12128746080,$$,4.5,7388,[],https://www.yelp.com/biz/levain-bakery-new-yor...
1,gooey-on-the-inside-new-york-4,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 40.72063, 'longitude': -73.99275}",(646) 972-0409,1694.634909,vPrJbdUxqCWVDDP5nQGWYA,https://s3-media3.fl.yelpcdn.com/bphoto/5Gbvc3...,False,"{'address1': '163 Chrystie St', 'address2': 'B...",Gooey on The Inside,+16469720409,$,4.0,101,"[pickup, delivery]",https://www.yelp.com/biz/gooey-on-the-inside-n...
2,schmackarys-new-york,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 40.760153883881, 'longitude': -73...",(646) 801-9866,6089.899672,jz_FnuRpCNzfwwHHQ6Nbeg,https://s3-media1.fl.yelpcdn.com/bphoto/pu2K85...,False,"{'address1': '362 W 45th St', 'address2': '', ...",Schmackary's,+16468019866,$,4.0,1786,"[pickup, delivery]",https://www.yelp.com/biz/schmackarys-new-york?...
3,chip-nyc-astoria,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...","{'latitude': 40.7655337630129, 'longitude': -7...",(917) 745-0101,9209.974859,TthttjDCINv6jOV28bEphg,https://s3-media3.fl.yelpcdn.com/bphoto/temzHc...,False,"{'address1': '30-06 34th St', 'address2': None...",Chip NYC,+19177450101,$$,4.0,502,[],https://www.yelp.com/biz/chip-nyc-astoria?adju...
4,levain-bakery-new-york-3,"[{'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 40.8049739, 'longitude': -73.9551...",(646) 455-0952,11553.808020,QHYqNQhQ8NVK0RX68Y4PuQ,https://s3-media1.fl.yelpcdn.com/bphoto/YMKN-H...,False,"{'address1': '2167 Frederick Douglass Blvd', '...",Levain Bakery,+16464550952,$$,4.5,650,[],https://www.yelp.com/biz/levain-bakery-new-yor...
5,chip-nyc-astoria-4,"[{'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 40.7755748, 'longitude': -73.9095...",(929) 522-0800,10525.026678,RgR0u3JQoVki3GC7n0Uc_Q,https://s3-media1.fl.yelpcdn.com/bphoto/OzMJM-...,False,"{'address1': '22-04 33rd St', 'address2': '', ...",Chip NYC,+19295220800,NaN,4.5,24,"[pickup, delivery]",https://www.yelp.com/biz/chip-nyc-astoria-4?ad...
6,the-cookie-connect-bloomfield-2,"[{'alias': 'desserts', 'title': 'Desserts'}]","{'latitude': 40.8366154, 'longitude': -74.1636...",(201) 822-5339,20401.641775,rbiGWxxFz1EWQiJnUAf3Pg,https://s3-media4.fl.yelpcdn.com/bphoto/Rt24YM...,False,"{'address1': '405 W Passaic Ave', 'address2': ...",The Cookie Connect,+12018225339,$$,4.5,137,[],https://www.yelp.com/biz/the-cookie-connect-bl...
7,city-cakes-new-york,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 40.74206, 'longitude': -73.99999}",(646) 688-2286,4099.618344,wQLNq8dQE4DxvUSQwzAagw,https://s3-media3.fl.yelpcdn.com/bphoto/-mDV2N...,False,"{'address1': '251 W 18th St', 'address2': '', ...",City Cakes,+16466882286,$$,4.5,589,[pickup],https://www.yelp.com/biz/city-cakes-new-york?a...
8,bens-cookies-new-york-4,"[{'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 40.732962, 'longitude': -73.99093}",(646) 590-1156,3072.862531,8GNA022E8X8LjjCstu4cyw,https://s3-media2.fl.yelpcdn.com/bphoto/8N6kuJ...,False,"{'address1': '822 Broadway', 'address2': '', '...",Ben's Cookies,+16465901156,$,3.5,163,"[pickup, delivery]",https://www.yelp.com/biz/bens-cookies-new-york...
9,bang-cookies-jersey-city-3,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...","{'latitude': 40.75567, 'longitude': -74.04607}",(888) 834-8885,7086.973427,yVAD1dPNat6mSsNWOskibA,https://s3-media4.fl.yelpcdn.com/bphoto/GJ8WvH...,False,"{'address1': '1183 Summit Ave', 'address2': No...",Bang Cookies,+18888348885,$,4.5,190,[],https://www.yelp.com/biz/bang-cookies-jersey-c...


In [17]:
df1['price_num'] = df1['price'].str.len()

print(df1.describe())
print('\n')
print(df1["rating"].corr(df1["review_count"]))
print('\n')
print(df1["price_num"].corr(df1["review_count"]))
print('\n')
print(df1["name"].unique())
print('\n')
print(df1["name"].value_counts())

           distance     rating  review_count  price_num
count     50.000000  50.000000     50.000000  47.000000
mean    7112.228624   4.140000    517.680000   1.659574
std     6715.930496   0.416986   1201.726402   0.522394
min      361.241192   3.000000      2.000000   1.000000
25%     3058.688660   4.000000     77.000000   1.000000
50%     4074.934782   4.000000    191.500000   2.000000
75%     8780.637439   4.500000    371.000000   2.000000
max    29575.852855   5.000000   7388.000000   3.000000


0.10156117844054108


0.12623217268834708


['Levain Bakery' 'Gooey on The Inside' "Schmackary's" 'Chip NYC'
 'The Cookie Connect' 'City Cakes' "Ben's Cookies" 'Bang Cookies'
 'Sofia & Grace Cookie' 'Milk & Cookies' 'One Girl Cookies'
 'Insomnia Cookies' 'Fat Cat Kitchen' 'Buttermilk Bakeshop'
 'Birdbath Neighborhood Green Bakery' 'Golden Glow Cookie Company'
 'The Good Batch' 'Ovenly' 'Woops! BakeShop' 'The Cookie Jar'
 'Dominique Ansel Bakery' 'Madonia Brothers Bakery'
 "La Bonbonniere's

In [18]:

df1.groupby(["name", "review_count"]).sum()

,,distance,is_closed,rating,price_num
name,review_count,,,,
Baked,292,1703.822384,False,4.0,2.0
Baking Mama,61,4623.493462,False,4.5,2.0
Bang Cookies,190,7086.973427,False,4.5,1.0
Ben's Cookies,163,3072.862531,False,3.5,1.0
Birdbath Neighborhood Green Bakery,234,2356.123963,False,4.0,2.0
Boomshakalaka NY,2,1030.521157,False,4.5,1.0
Brooklyn Sweet Spot,69,2363.025112,False,4.0,2.0
Butter & Scotch,446,4982.532803,False,4.0,2.0
Buttermilk Bakeshop,119,3554.276424,False,4.5,2.0


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [19]:
coordinates = df1["coordinates"].to_dict()
coordinates_list = []
for key, values in coordinates.items():
    coordinates_list.append(values)
coordinates_list

[{'latitude': 40.7799404643263, 'longitude': -73.980282552649},
 {'latitude': 40.72063, 'longitude': -73.99275},
 {'latitude': 40.760153883881, 'longitude': -73.990944257291},
 {'latitude': 40.7655337630129, 'longitude': -73.9192518515278},
 {'latitude': 40.8049739, 'longitude': -73.9551513},
 {'latitude': 40.7755748, 'longitude': -73.9095839},
 {'latitude': 40.8366154, 'longitude': -74.1636902},
 {'latitude': 40.74206, 'longitude': -73.99999},
 {'latitude': 40.732962, 'longitude': -73.99093},
 {'latitude': 40.75567, 'longitude': -74.04607},
 {'latitude': 40.6844899, 'longitude': -73.92289},
 {'latitude': 40.731812157368, 'longitude': -74.00443554356},
 {'latitude': 40.687328, 'longitude': -73.990465},
 {'latitude': 40.75009, 'longitude': -73.94062},
 {'latitude': 40.70377, 'longitude': -74.0104099},
 {'latitude': 40.781532, 'longitude': -73.979286},
 {'latitude': 40.73314, 'longitude': -73.98607},
 {'latitude': 40.6749461087063, 'longitude': -73.9816744176426},
 {'latitude': 40.72596,

In [20]:
names = df1["name"].tolist()
names

['Levain Bakery',
 'Gooey on The Inside',
 "Schmackary's",
 'Chip NYC',
 'Levain Bakery',
 'Chip NYC',
 'The Cookie Connect',
 'City Cakes',
 "Ben's Cookies",
 'Bang Cookies',
 'Sofia & Grace Cookie',
 'Milk & Cookies',
 'One Girl Cookies',
 'Chip NYC',
 'Insomnia Cookies',
 'Levain Bakery',
 'Fat Cat Kitchen',
 'Buttermilk Bakeshop',
 'Birdbath Neighborhood Green Bakery',
 'Golden Glow Cookie Company',
 'The Good Batch',
 'Ovenly',
 'Woops! BakeShop',
 'The Cookie Jar',
 'Dominique Ansel Bakery',
 'Insomnia Cookies',
 'Insomnia Cookies',
 'Ovenly',
 'Madonia Brothers Bakery',
 "La Bonbonniere's Cookie Jar Bakery",
 'Sweet Corner Bakeshop',
 'Jacques Torres Chocolate',
 'Butter & Scotch',
 "Sweet Sweet Sue's Cupcake Cafe",
 'Mah-Ze-Dahr Bakery',
 'Baking Mama',
 "Orwasher's",
 'One Girl Cookies',
 'Krumville Bake Shop',
 'Two Little Red Hens',
 'Brooklyn Sweet Spot',
 'Sugar & Water NYC',
 'Insomnia Cookies',
 'Ovenly',
 'Boomshakalaka NY',
 'Baked',
 'Insomnia Cookies',
 "Mia's Bakery

In [21]:
import folium
manhattan_map = folium.Map(location=[40.7589, -73.9851], zoom_start=11)
manhattan_map

In [22]:
points = []
for i in range(len(coordinates_list)):
    x = coordinates_list[i]["latitude"]
    y = coordinates_list[i]["longitude"]
    point = (x, y)
    points.append(point)
for p in points:
    lat = p[0]
    long = p[1]
    marker = folium.Marker(location=[lat, long])
    marker.add_to(manhattan_map)
manhattan_map

In [24]:
for i, p in enumerate(points):
    lat = p[0]
    long = p[1]
    name = names[i]
    popup_text = "Lattitude: {}, Longitude: {}".format(name,lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(manhattan_map)
manhattan_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!